In [1]:
import sys
sys.path.append("D:/StateOfStates/")

import pandas as pd
import addfips
import Partisian as Partisian
import CauseOfDeathData
from LifeData import LifeData
import numpy as np
import json
import psycopg2
import json
import CauseOfDeathData

### Setup DB connection

In [2]:
def get_db_properties():
    props = {}
    separator = "="
            
    with open(r"C:\Users\zeusg\OneDrive\Documents\Github\habit_final\database.properties") as file:
        for line in file: 
            if separator in line:
                name, value = line.split(separator, 1)
                props[name.strip()] = value.strip()
    return props
db_prop = get_db_properties()
connection = psycopg2.connect(
    host=db_prop["host"],
    user=db_prop["username"],
    password=db_prop["password"],
    database="StateOfStates",
)

### Helper Dicts to convert states->abv->fips

In [3]:
states =  {'Alaska': 'AK','Alabama': 'AL','Arkansas': 'AR','American Samoa': 'AS','Arizona': 'AZ','California': 'CA','Colorado': 'CO','Connecticut': 'CT','District of Columbia': 'DC','Delaware': 'DE','Florida': 'FL','Georgia': 'GA','Guam': 'GU','Hawaii': 'HI','Iowa': 'IA','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Massachusetts': 'MA','Maryland': 'MD','Maine': 'ME','Michigan': 'MI','Minnesota': 'MN','Missouri': 'MO','Northern Mariana Islands': 'MP','Mississippi': 'MS','Montana': 'MT','National': 'NA','North Carolina': 'NC','North Dakota': 'ND','Nebraska': 'NE','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','Nevada': 'NV','New York': 'NY','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA','Puerto Rico': 'PR','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Virginia': 'VA','Virgin Islands': 'VI','Vermont': 'VT','Washington': 'WA','Wisconsin': 'WI','West Virginia': 'WV','Wyoming': 'WY'}
states_abv = {v: k for k, v in states.items()}
af = addfips.AddFIPS()


### Load Partisian DF

In [4]:
#State Control
partisian_df = Partisian.get_state_part_score_all()
partisian_df = partisian_df.rename(columns={"State": "State_Code"})
#State Names
states_df = partisian_df[["STATE", "State_Code", "fips"]].drop_duplicates().copy()
states_df["level"] = "State"
states_df = states_df.rename(columns={"STATE": "name", "State_Code": "acronym"})
states_df["parent"] = 1
states_df = states_df[["level", "name", "parent", "fips", "acronym"]]
#County Votes
county_votes_df = Partisian.get_county_votes()
## TODO - Clean up county votes




In [5]:
def insert_states(df):
    cursor = connection.cursor()
    for index, row in df.iterrows():
        cursor.execute("INSERT INTO public.locations (level, name, parent, fips, acronym) \
                       VALUES (%s, %s, %s, %s, %s);", 
                       (row["level"], row["name"], row["parent"], row["fips"], row["acronym"]))
    connection.commit()
    cursor.close()
    return "success"
#insert_states(states_df)


In [6]:
def insert_partisian_control(df):
    cursor = connection.cursor()
    for index, row in df.iterrows():
        cursor.execute("SELECT id FROM public.locations WHERE acronym = %s;", (row["State_Code"],))
        locations_fk = cursor.fetchone()[0]
        cursor.execute("INSERT INTO public.partisans (year, control_score, locations_id_fkey) \
            VALUES (%s, %s, %s);", 
            (row["Year"], row["Part_Score"], locations_fk))
    connection.commit()
    cursor.close()
    return "success"
#insert_partisian_control(partisian_df)

### Load ICD10 codes

In [7]:
icd10 = CauseOfDeathData.get_heirarchy_df().T.reset_index()
icd10 = icd10.rename({"index": "name"}, axis=1)

### Insert ICD10 and Hierarchy structure

In [8]:
def insert_inital_icd10(df):
    cursor = connection.cursor()
    for index, row in df.iterrows():
        cursor.execute("INSERT INTO public.icd_10_codes (name, common_name) \
            VALUES (%s, %s);", 
            (row["name"], row["common_name"]))
    connection.commit()
    cursor.close()
    return "success"
def add_icd10_hierarchy(df):
    cursor = connection.cursor()
    for index, row in df.iterrows():
        if len(row["parents"]) > 0:
            cursor.execute("SELECT id FROM public.icd_10_codes WHERE name = %s;", (row["parents"][0],))
            icd10_fk = cursor.fetchone()[0]
            cursor.execute("UPDATE public.icd_10_codes SET parent = %s WHERE name = %s;", 
                (icd10_fk, row["name"]))
    connection.commit()
    cursor.close()
    return "success"

def add_icd10_codes(df):
    cursor = connection.cursor()
    for index, row in df.iterrows():
        codes_json = json.dumps(row["ranges"])
        cursor.execute("UPDATE public.icd_10_codes SET codes = %s WHERE name = %s;", 
            (codes_json, row["name"]))
    connection.commit()
    cursor.close()
    ## TODO - Cleaning needed when code spans chapters IE: A00-B99
    return "success"
    
#insert_inital_icd10(icd10)
#add_icd10_hierarchy(icd10)
#add_icd10_codes(icd10)

### Load Cause Of Death for all states

In [9]:
cod = CauseOfDeathData.load_state_cod(state="All")
cod["year"] = 2019
cod = cod[["Rate", "Deaths", "Population","year","ICD-10 113 Cause List","State","Ten-Year Age Groups Code", "Crude Rate"]]
cod = cod[cod["State"] != "District of Columbia"]

In [14]:
def add_cod(df):
    try:
        cursor = connection.cursor()
        for index, row in df.iterrows():
            cursor.execute("SELECT id FROM public.locations WHERE name = %s;", (row["State"],))
            state_id = cursor.fetchone()[0]
            cursor.execute("SELECT id FROM public.icd_10_codes WHERE name = %s;", (row["ICD-10 113 Cause List"],))
            icd10_id = cursor.fetchone()[0]
            if row["Ten-Year Age Groups Code"] == "1":
                start_age = 0
                end_age = 1
            elif row["Ten-Year Age Groups Code"] == "85+":
                start_age = 85
                end_age = 200
            else:
                start_age = int(row["Ten-Year Age Groups Code"].split("-")[0])
                end_age = int(row["Ten-Year Age Groups Code"].split("-")[1])
            unreliable = False
            if row["Crude Rate"] == "Unreliable":
                unreliable = True
            cursor.execute("INSERT INTO public.causes_of_death (rate, deaths, population, year, locations_id_fkey, icd_10_codes_id_fkey, start_age, end_age, unreliable) \
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);", 
                (row["Rate"], row["Deaths"], row["Population"], row["year"], state_id, icd10_id, start_age, end_age, unreliable))

        connection.commit()
    except Exception as e:
        print(row["State"])
        print(e)
    finally:
        cursor.close()
    return "success"
add_cod(cod)

'success'

In [18]:
sql = "select l.name as state_name, l.acronym, l.fips, \
cod.rate, cod.deaths, cod.population, cod.start_age, cod.end_age, \
ic.name, ic.common_name, ic.parent, \
ic2.name as parent_name, ic2.common_name as parent_common_name \
from causes_of_death cod \
inner join icd_10_codes ic on cod.icd_10_codes_id_fkey = ic.id \
left join icd_10_codes ic2 on ic.parent = ic2.id \
inner join locations l ON cod.locations_id_fkey = l.id \
where l.level = 'State'"

In [19]:
df = pd.read_sql(sql, connection)

C:\Users\zeusg\AppData\Local\Temp/ipykernel_17484/1685767199.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, connection)


In [20]:
df

,state_name,acronym,fips,rate,deaths,population,start_age,end_age,name,common_name,parent,parent_name,parent_common_name
0,Alabama,AL,1,17.57,10,56901,0,1,"Certain other intestinal infections (A04,A07-A09)",Certain other intestinal infections,NaN,None,None
1,Alabama,AL,1,1.52,10,657667,55,64,"Certain other intestinal infections (A04,A07-A09)",Certain other intestinal infections,NaN,None,None
2,Alabama,AL,1,5.18,26,501447,65,74,"Certain other intestinal infections (A04,A07-A09)",Certain other intestinal infections,NaN,None,None
3,Alabama,AL,1,12.07,31,256847,75,84,"Certain other intestinal infections (A04,A07-A09)",Certain other intestinal infections,NaN,None,None
4,Alabama,AL,1,27.31,25,91543,85,200,"Certain other intestinal infections (A04,A07-A09)",Certain other intestinal infections,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23225,Washington,WA,53,1.26,13,1028141,35,44,"#Legal intervention (Y35,Y89.0)",Legal intervention,102.0,Other and unspecified events of undetermined i...,Unknown Intent (Other)
23226,North Carolina,NC,37,0.72,10,1387060,15,24,Accidental discharge of firearms (W32-W34),Accident (Guns),83.0,"Nontransport accidents (W00-X59,Y86)",Nontransport accidents
23227,Texas,TX,48,0.37,15,4055517,15,24,Accidental discharge of firearms (W32-W34),Accident (Guns),83.0,"Nontransport accidents (W00-X59,Y86)",Nontransport accidents
23228,Texas,TX,48,0.31,10,3275242,55,64,Other tuberculosis (A17-A19),Other tuberculosis,103.0,#Tuberculosis (A16-A19),#Tuberculosis


In [22]:
old_df = CauseOfDeathData.load_state_cod(state="All")
old_df

,State,State Code,ICD-10 113 Cause List,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,Child_Adj_Rate,common_name,children,parents
0,Alabama,1.0,"Certain other intestinal infections (A04,A07-A09)",< 1 year,1,10.0,56901.0,Unreliable,17.57,17.57,Certain other intestinal infections,[#Enterocolitis due to Clostridium difficile (...,[]
1,Alabama,1.0,"Certain other intestinal infections (A04,A07-A09)",55-64 years,55-64,10.0,657667.0,Unreliable,1.52,1.52,Certain other intestinal infections,[#Enterocolitis due to Clostridium difficile (...,[]
2,Alabama,1.0,"Certain other intestinal infections (A04,A07-A09)",65-74 years,65-74,26.0,501447.0,5.2,5.18,5.18,Certain other intestinal infections,[#Enterocolitis due to Clostridium difficile (...,[]
3,Alabama,1.0,"Certain other intestinal infections (A04,A07-A09)",75-84 years,75-84,31.0,256847.0,12.1,12.07,12.07,Certain other intestinal infections,[#Enterocolitis due to Clostridium difficile (...,[]
4,Alabama,1.0,"Certain other intestinal infections (A04,A07-A09)",85+ years,85+,25.0,91543.0,27.3,27.31,27.31,Certain other intestinal infections,[#Enterocolitis due to Clostridium difficile (...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23420,Washington,53.0,"#Legal intervention (Y35,Y89.0)",35-44 years,35-44,13.0,1028141.0,Unreliable,1.26,1.26,Legal intervention,[],[Other and unspecified events of undetermined ...
23421,North Carolina,37.0,Accidental discharge of firearms (W32-W34),15-24 years,15-24,10.0,1387060.0,Unreliable,0.72,0.72,Accident (Guns),[],"[Nontransport accidents (W00-X59,Y86)]"
23422,Texas,48.0,Accidental discharge of firearms (W32-W34),15-24 years,15-24,15.0,4055517.0,Unreliable,0.37,0.37,Accident (Guns),[],"[Nontransport accidents (W00-X59,Y86)]"
23423,Texas,48.0,Other tuberculosis (A17-A19),55-64 years,55-64,10.0,3275242.0,Unreliable,0.31,0.31,Other tuberculosis,[],[#Tuberculosis (A16-A19)]
